In [1]:
# Import Libraries
import pandas as pd 
import matplotlib.pyplot as plt
import os
from datetime import datetime
import time

In [2]:
file = 'Solar_installation_data.csv'

solar = pd.read_csv(file)
solar.head()

,Unnamed: 0,Project Number,City,Zip Code,Sector,Electric Utility,Purchase Type,Project Status,Date Application Received,Date Completed,Project Cost,$Incentive,Total Nameplate kW DC,Expected KWh Annual Production,Contractor,Primary PV Module Manufacturer,Location 1,$_per_watt,$_per_watt_wincentives
0,0,4260-88892,Three Mile Bay,13693,Residential,National Grid,Purchase,Complete,2/1/2016,2016-09-21,29680.00,4240.0,8.80,10330,"Fourth Coast, Inc",SolarWorld,"Three Mile\nBay, NY 13693\n",3.372727,2.890909
1,2,89350,Queens,11434,Residential,Consolidated Edison,Lease,Complete,5/30/2017,2017-07-25,30791.25,2898.0,7.25,8504,SunPower Capital LLC,SunPower,"Queens, NY 11434\n(40.676003, -73.775656)",4.247069,3.847345
2,3,93198,Bronx,10456,Residential,Consolidated Edison,Lease,Complete,7/19/2017,2017-09-20,13906.00,1848.0,4.62,5423,SunRun Inc.,LG Electronics,"Bronx, NY 10456\n(40.830652, -73.908566)",3.009957,2.609957
3,7,2011-157041-SLPR,Old Field,11733,Residential,PSEG Long Island,Purchase,Complete,10/9/2005,2005-10-09,NaN,14000.0,3.50,4108,NaN,NaN,"Old Field, NY 11733\n(40.93232, -73.105)",NaN,NaN
4,8,2011-158360-SLPR,Medford,11763,Residential,PSEG Long Island,Purchase,Complete,4/23/2009,2009-04-23,136952.00,35000.0,10.25,12032,NaN,NaN,"Medford, NY 11763\n(40.822436, -72.982416)",13.361171,9.946537


In [3]:
#Convert Date variables to datetime
solar['Date Completed'] = pd.to_datetime(solar['Date Completed'])
solar['Date Application Received'] = pd.to_datetime(solar['Date Application Received'])

#Convert datetimes to unix timestamps
def convert_time(time_dt):
    return int(time.mktime(time_dt.timetuple()))

#print columns with unix ts and length of project in days
solar['Date Application Received Int'] = solar.apply(lambda row: convert_time(row['Date Application Received']), axis=1)
solar['Date Completed Int'] = solar.apply(lambda row: convert_time(row['Date Completed']), axis=1)
solar['Project Days Int'] = solar.apply(lambda row: int((row['Date Completed Int']-row['Date Application Received Int'])/86400), axis=1)

#parse the datetime objects
solar['Completed Year'] = solar.apply(lambda row: row['Date Completed'].year, axis=1)
solar['Completed Month'] = solar.apply(lambda row: row['Date Completed'].month, axis=1)

#Create additional variables
solar['$_per_watt'] = round(solar['Project Cost'] / solar['Total Nameplate kW DC'] / 1000, 2)
solar['$_per_watt_wincentives'] = round((solar['Project Cost'] - solar['$Incentive'] ) / solar['Total Nameplate kW DC'] /1000, 2)



solar.head()

,Unnamed: 0,Project Number,City,Zip Code,Sector,Electric Utility,Purchase Type,Project Status,Date Application Received,Date Completed,...,Contractor,Primary PV Module Manufacturer,Location 1,$_per_watt,$_per_watt_wincentives,Date Application Received Int,Date Completed Int,Project Days Int,Completed Year,Completed Month
0,0,4260-88892,Three Mile Bay,13693,Residential,National Grid,Purchase,Complete,2016-02-01,2016-09-21,...,"Fourth Coast, Inc",SolarWorld,"Three Mile\nBay, NY 13693\n",3.37,2.89,1454313600,1474441200,232,2016,9
1,2,89350,Queens,11434,Residential,Consolidated Edison,Lease,Complete,2017-05-30,2017-07-25,...,SunPower Capital LLC,SunPower,"Queens, NY 11434\n(40.676003, -73.775656)",4.25,3.85,1496127600,1500966000,56,2017,7
2,3,93198,Bronx,10456,Residential,Consolidated Edison,Lease,Complete,2017-07-19,2017-09-20,...,SunRun Inc.,LG Electronics,"Bronx, NY 10456\n(40.830652, -73.908566)",3.01,2.61,1500447600,1505890800,63,2017,9
3,7,2011-157041-SLPR,Old Field,11733,Residential,PSEG Long Island,Purchase,Complete,2005-10-09,2005-10-09,...,NaN,NaN,"Old Field, NY 11733\n(40.93232, -73.105)",NaN,NaN,1128841200,1128841200,0,2005,10
4,8,2011-158360-SLPR,Medford,11763,Residential,PSEG Long Island,Purchase,Complete,2009-04-23,2009-04-23,...,NaN,NaN,"Medford, NY 11763\n(40.822436, -72.982416)",13.36,9.95,1240470000,1240470000,0,2009,4


In [4]:
# Save df as csv
output_file = 'project_data_final.csv'
solar.to_csv(output_file)